In [ ]:
import pandas as pd
import sqlalchemy as sql
import datetime
import pymysql

# read password and user to database
credentials_file = r'./credentials.pw'

credentials = pd.read_csv(credentials_file, header=0)
user = credentials['username'].values[0]
pw = credentials['password'].values[0]

host="twofast-rpi3-0"  # your host
user='reader' # username
passwd='heiko'  # password
db="NG_twofast_DB" # name of the database
connect_string = 'mysql+pymysql://%(user)s:%(pw)s@%(host)s:3306/%(db)s'% {"user": user, "pw": pw, "host": host, "db": db}
sql_engine = sql.create_engine(connect_string)



def truncateTable(tableName):
    sql_engine.execute("TRUNCATE TABLE %(tbl)s" % {"tbl": tableName})
    

def switchDLLstateInControlTable(sql_engine):
    timeExecuted = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    sql_engine.execute("UPDATE microwave_generator_control SET DLL_on = 1 - DLL_on, time = '%(timeExecuted)s' WHERE id = 1" % {"timeExecuted": timeExecuted})

def initializeDLLstateValue(sql_engine):
    sql_engine.execute("INSERT INTO microwave_generator_control (DLL_on) VALUES (0)")
    
def getState(sql_engine):
    """
    Reads the last 10 entries of the two temperatures and returns it in a dataframe
    """
    query = "SELECT * FROM microwave_generator_state ORDER BY id DESC LIMIT 10"
    df = pd.read_sql(query, sql_engine)

    # columns: time (timestamp), relais_5 (tinyint), relais_24 (tinyint), rf_status (tinyint), id (primary key)
    return df

In [5]:
import pandas as pd
import sqlalchemy as sql
import datetime
import pymysql

host="twofast-rpi3-0"  # your host
user='reader' # username
pw='heiko'  # password
db="NG_twofast_DB" # name of the database
connect_string = 'mysql+pymysql://%(user)s:%(pw)s@%(host)s:3306/%(db)s'% {"user": user, "pw": pw, "host": host, "db": db}
sql_engine = sql.create_engine(connect_string)

def insert_to_db(sql_engine, temp, humid):
    sql_engine.execute("INSERT INTO temp_humid_sensor (temp, humid) VALUES (%(temp)s, %(humid)s)" % {"temp":temp, "humid":humid})

temp = 10
humid = 11
insert_to_db(sql_engine, temp, humid)